# Sistema di produzione di neve artificiale
---

## Formulazione del problema

- orizzonte temporale: $t=0,1,...T$, $T = 72$ ore considerando un time step orario
- stato $x_t: (temperatura\_ora\_t,umidità\_ora\_t,livello\_neve\_ora\_t) = (T_t,RH_t,d_t) = (i,j,k)$
- spazio degli stati $x_t \in X_t=\{(i,j,k): i=1,...,50, j=1,...,50, k=1,...,n\}$
    - $T_t \in\{T_1,...,T_{50}\}_t$ con valori diversi da ora in ora
        - $T_t$ variabile aleatoria discreta con pmf $P(T_t=T_i)_t$
    - $RH_t \in\{RH_1,...,RH_{50}\}_t$ con valori diversi da ora in ora
        - $RH_t$ variabile aleatoria discreta con pmf $P(RH_t=RH_i)_t$
    - $d_t \in D=\{0,...,max_d\}, |D|=n$
- ingresso $u_t=(af_t,wf_t)$
    - $af_t$: flusso di aria iniettato nella macchina sparaneve nell'ora t $[m^3/h]$
    - $wf_t$: flusso di acqua iniettato nella macchina sparaneve nell'ora t $[m^3/h]$
- spazio degli ingressi $u_t \in U= 
    \begin{cases}
    \{(0,0)\} & \text{se } \text{la } \text{neve } \text{non } \text{può } \text{essere } \text{prodotta } \text{dati } (T_t,RH_t) \\
    \{(min_{af},min_{wf}),...,(max_{af},max_{wf})\} & \text{altrimenti } 
    \end{cases}$
    - $min_{wf} = 0.6 [m^3/h], max_{wf} = [3.6,5.4] [m^3/h]$
        - step di discretizzazione: $0.2$
    - $min_{af} = 6 [m^3/h], max_{af} = 27 [m^3/h]$
        - step di dicretizzazione: $1$
    - la neve non può essere prodotta se la temperatura del bulbo umido è superiore alla soglia operativa $T_{wb,soglia} = -2.5 °C$
- matrice di transizione per le componenti di temperatura e umidità dello stato
    - $P((T_{t+1}=i,RH_{t+1}=j)|(T_{t}=i',RH_{t}=j'))=P(T_{t+1}=i|T_t=i')_{t+1}*P(RH_{t+1}=j|RH_t=j')_{t+1}$
- funzione di transizione per la componente "livello_neve" dello stato
    - $d_{t+1}=d_t+Neve\_prodotta(x_t,u_t)-Neve\_fusa(x_t)$
    - valore da approssimare al valore discreto più vicino assunto da $d_t$
- funzione per determinare la neve prodotta in un time step in $m^3$
    - $Neve\_prodotta(T_t,RH_t,af_t,wf_t)=k(T_t,RH_t)*min(af_t,wf_t)$
    - $k=\alpha * max(0,T_{wb,soglia}-T_{wb})$ coefficiente di efficienza
    - $\alpha:$ coefficiente calibrato $\in [0.4,1]$
    - $T_{wb,soglia} = -2.5 °C$: soglia (wet-bulb) operativa del generatore
    - $T_{wb} ≈ T_t ⋅arctan(0.151977⋅(RH_t+8.313659)^{0.5})+arctan(T_t+RH_t)-arctan(RH_t−1.676331)
+0.00391838⋅RH^{1.5}⋅arctan(0.023101⋅RH)−4.686035$
        - dove $T_t$ in $°C$ e $RH_t$ in percentuale
- funzione per determinare la neve fusa in un time step in $m^3$
    - $Neve\_fusa(x_t)= (DDF/(1000*24)) * max(T_{avg}-T_0,0) * A$
        - $DDF = degree-day-factor (mm/°C/giorno)$ (tipicamente 1–10 mm)
        - $T_{avg} = temperatura-media °C$
        - $T_0 = soglia (≈°C)$
        - $A = area (m^2)$
- stage cost $g(x_t,u_t):$ deterministico, tempo-invariante
    - $g(x_t,u_t)=prezzo\_energia*(E_{ventola}+E_{compressore}(u_t)+E_{pompa}(u_t))$
    - $prezzo\_energia$ in kWh
    - $E_{ventola}=18.5 kWh$ costante
    - $E_{compressore}(u_t)= e * af_t * 60$
        - $e=6.2 [kW/(m^3/min)]$
        - $af_t [m^3/h]$
    - $E_{pompa}(u_t)=k_{pump}*wf_t$
        - $k_pump=3.5 [kWh/m^3]$
        - $af_t [m^3/h]$
- Terminal cost $g_T(x_T)$:
    - $g_T((i,j,k)) = \begin{cases} 0 & \text{if } k = n \\ \infty & \text{if } k \neq n \end{cases}$
    - costo infinito se non viene prodotta $max_d [m^3]$ di neve al termine dell'orizzonte temporale

## Importazione moduli

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import math 
from functions import *

## Parametri del problema

In [2]:
n_fasce=24
max_d=100
step_d=0.5
min_af=6
min_wf=0.6
max_af=27
max_wf=4.5 #[3.6,5.4]
af_step=1
wf_step=0.2

## Formulazione del problema e parametri

In [3]:
# caricamento dati temperatura e umidità da file
with open("pmf_temperature.json", "r", encoding="utf-8") as f1:
    temperature = json.load(f1)

with open("pmf_humidity.json", "r", encoding="utf-8") as f2:
    humidity = json.load(f2)

temp_val=[]
temp_pmf=[]
humid_val=[]
humid_pmf=[]
for fasce,dict_val in temperature.items():
    temp_val.append(np.array(dict_val['values']))
    temp_pmf.append(np.array(dict_val['pmf']))
for fasce,dict_val in humidity.items():
    humid_val.append(np.array(dict_val['values']))
    humid_pmf.append(np.array(dict_val['pmf']))

#numero di valori di temperatura e umidità per fascia
n_val=temp_val[0].shape[0]

depth_space=np.arange(0,max_d+step_d,step_d)
num_d_val=depth_space.shape[0]

# spazio degli stati
state_space=[]

for f in range(n_fasce):
    X=np.empty((n_val,n_val,num_d_val,3))
    for i,t in enumerate(temp_val[f]):
        for j,u in enumerate(humid_val[f]):
            for k,d in enumerate(depth_space):            
                X[i,j,k]=[t,u,d]
    state_space.append(X)

#spazio degli ingressi
U=input_space(-1,90,min_af, max_af, min_wf, max_wf, af_step, wf_step)

#calcolo matrice P
P_list = []

n_T = temp_val[0].shape[0]
n_RH = humid_val[0].shape[0]
num_states = n_T * n_RH

for f in range(n_fasce - 1):

    # pmf nella fascia successiva
    pmf_T = temp_pmf[f+1]      # shape (n_T,)
    pmf_RH = humid_pmf[f+1]    # shape (n_RH,)

    # pmf congiunta temperatura-umidità
    joint_TR = np.outer(pmf_T, pmf_RH)   # shape (n_T, n_RH)

    # flatten (ordine: T varia lentamente, RH veloce)
    flat_next = joint_TR.reshape(-1)     # shape (num_states,)

    # matrice di transizione (tutte le righe uguali)
    P = np.tile(flat_next, (num_states, 1))  # shape (num_states, num_states)

    P_list.append(P)


In [4]:
print(X[0,0,0])

[-16.70485     24.60478088   0.        ]


In [5]:
print(wet_bulb_temperature(-1,95))

-1.5343242442495588


In [6]:
print(P_list[0])

[[4.11522634e-05 4.11522634e-05 0.00000000e+00 ... 6.00137174e-04
  4.11522634e-04 8.57338820e-05]
 [4.11522634e-05 4.11522634e-05 0.00000000e+00 ... 6.00137174e-04
  4.11522634e-04 8.57338820e-05]
 [4.11522634e-05 4.11522634e-05 0.00000000e+00 ... 6.00137174e-04
  4.11522634e-04 8.57338820e-05]
 ...
 [4.11522634e-05 4.11522634e-05 0.00000000e+00 ... 6.00137174e-04
  4.11522634e-04 8.57338820e-05]
 [4.11522634e-05 4.11522634e-05 0.00000000e+00 ... 6.00137174e-04
  4.11522634e-04 8.57338820e-05]
 [4.11522634e-05 4.11522634e-05 0.00000000e+00 ... 6.00137174e-04
  4.11522634e-04 8.57338820e-05]]


In [7]:
print(sum(P_list[0][0]))

1.0000000000000016


In [8]:
print(P_list[1])

[[3.42935528e-05 0.00000000e+00 0.00000000e+00 ... 6.79012346e-04
  3.90946502e-04 1.23456790e-04]
 [3.42935528e-05 0.00000000e+00 0.00000000e+00 ... 6.79012346e-04
  3.90946502e-04 1.23456790e-04]
 [3.42935528e-05 0.00000000e+00 0.00000000e+00 ... 6.79012346e-04
  3.90946502e-04 1.23456790e-04]
 ...
 [3.42935528e-05 0.00000000e+00 0.00000000e+00 ... 6.79012346e-04
  3.90946502e-04 1.23456790e-04]
 [3.42935528e-05 0.00000000e+00 0.00000000e+00 ... 6.79012346e-04
  3.90946502e-04 1.23456790e-04]
 [3.42935528e-05 0.00000000e+00 0.00000000e+00 ... 6.79012346e-04
  3.90946502e-04 1.23456790e-04]]
